# Assignment 3 Part 1: Developing Your Own Classifier

In [1]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
os.chdir("/content/drive/My Drive/A-Courses-PhD/Term_17_Fall2020/CS498-DL/assignments/Project/Explainable_GTSRB")
#os.chdir("/content/drive/My Drive/CS498-DL/assignments/Assignment_03/My_assignment3_p1_starterkit")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch==1.4.0 torchvision==0.5.0


In [2]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision

import PIL
from torchvision import transforms
from sklearn.metrics import average_precision_score
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
#from kaggle_submission import output_submission_csv
from classifier import SimpleClassifier, Classifier, Classifier_moreConv#, AlexNet
#from voc_dataloader import VocDataset, VOC_CLASSES
from dataloader import MyDataset, My_classes

from sklearn.model_selection import train_test_split

%matplotlib inline
%load_ext autoreload
%autoreload 2

path ='/content/GTSRB_data/'
#path = 'G:/My Drive/A-Courses-PhD/Term_17_Fall2020/CS498-DL/assignments/Project/gtsrb-german-traffic-sign/GTSRB_data'
num_classes = len(My_classes)

In [4]:
if 0:
  import shutil 
  shutil.copyfile("GTSRB_data.tar", "/content/GTSRB_data.tar")
  !tar -xf "/content/GTSRB_data.tar" -C "/content/" 
  #shutil.move("/content/VOCdevkit/", "/content/VOCdevkit_2007")


# Part 1B: Design your own network

In this notebook, your task is to create and train your own model for multi-label classification on VOC Pascal.

## What to do
1. You will make change on network architecture in ```classifier.py```.
2. You may also want to change other hyperparameters to assist your training to get a better performances. Hints will be given in the below instructions.

## What to submit
Check the submission template for details what to submit. 

In [3]:
def train_classifier(train_loader, classifier, criterion, optimizer):
    classifier.train()
    loss_ = 0.0
    losses = []
    for i, (images, labels, _) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = classifier(images)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        losses.append(loss)
    return torch.stack(losses).mean().item()

In [4]:
def test_classifier(test_loader, classifier, criterion, print_ind_classes=True, print_total=True):
    classifier.eval()
    losses = []
    with torch.no_grad():
        y_true = np.zeros((0,num_classes))
        y_score = np.zeros((0,num_classes))
        for i, (images, labels, _) in enumerate(test_loader):
            images, labels = images.to(device), labels.to(device)
            logits = classifier(images)
            y_true = np.concatenate((y_true, labels.cpu().numpy()), axis=0)
            y_score = np.concatenate((y_score, logits.cpu().numpy()), axis=0)
            loss = criterion(logits, labels)
            losses.append(loss.item())
        aps = []
        # ignore first class which is background
        for i in range(1, y_true.shape[1]):
            ap = average_precision_score(y_true[:, i], y_score[:, i])
            if print_ind_classes:
                print('-------  Class: {:<12}     AP: {:>8.4f}  -------'.format(My_classes[i], ap))
            aps.append(ap)
        
        mAP = np.mean(aps)
        test_loss = np.mean(losses)
        if print_total:
            print('mAP: {0:.4f}'.format(mAP))
            print('Avg loss: {}'.format(test_loss))
        
    return mAP, test_loss, aps

In [5]:
def plot_losses(train, val, test_frequency, num_epochs):
    plt.plot(train, label="train")
    indices = [i for i in range(num_epochs) if ((i+1)%test_frequency == 0 or i ==0)]
    plt.plot(indices, val, label="val")
    plt.title("Loss Plot")
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.legend()
    plt.show()
    
def plot_mAP(train, val, test_frequency, num_epochs):
    indices = [i for i in range(num_epochs) if ((i+1)%test_frequency == 0 or i ==0)]
    plt.plot(indices, train, label="train")
    plt.plot(indices, val, label="val")
    plt.title("mAP Plot")
    plt.ylabel("mAP")
    plt.xlabel("Epoch")
    plt.legend()
    plt.show()
    

In [6]:

def train(classifier, num_epochs, train_loader, val_loader, criterion, optimizer, test_frequency=5):
    train_losses = []
    train_mAPs = []
    val_losses = []
    val_mAPs = []
    decayRate = 0.96
    

    for epoch in range(1,num_epochs+1):
        print("Starting epoch number " + str(epoch))
        train_loss = train_classifier(train_loader, classifier, criterion, optimizer)
        train_losses.append(train_loss)
        #lr_scheduler.step()
        #print('learning rate :', get_lr(lr_scheduler.optimizer))

        print("Loss for Training on Epoch " +str(epoch) + " is "+ str(train_loss))
        if(epoch%test_frequency==0 or epoch==1):
            mAP_train, _, _ = test_classifier(train_loader, classifier, criterion, False, False)
            train_mAPs.append(mAP_train)
            mAP_val, val_loss, _ = test_classifier(val_loader, classifier, criterion)
            print('Evaluating classifier')
            print("Mean Precision Score for Testing on Epoch " +str(epoch) + " is "+ str(mAP_val))
            val_losses.append(val_loss)
            val_mAPs.append(mAP_val)
    
    return classifier, train_losses, val_losses, train_mAPs, val_mAPs

# Developing Your Own Model

### Goal
To meet the benchmark for this assignment you will need to improve the network. Note you should have noticed pretrained Alenxt performs really well, but training Alexnet from scratch performs much worse. We hope you can design a better architecture over both the simple classifier and AlexNet to train from scratch.

### How to start
You may take inspiration from other published architectures and architectures discussed in lecture. However, you are NOT allowed to use predefined models (e.g. models from torchvision) or use pretrained weights. Training must be done from scratch with your own custom model.

#### Some hints
There are a variety of different approaches you should try to improve performance from the simple classifier:

* Network architecture changes
    * Number of layers: try adding layers to make your network deeper
    * Batch normalization: adding batch norm between layers will likely give you a significant performance increase
    * Residual connections: as you increase the depth of your network, you will find that having residual connections like those in ResNet architectures will be helpful
* Optimizer: Instead of plain SGD, you may want to add a learning rate schedule, add momentum, or use one of the other optimizers you have learned about like Adam. Check the `torch.optim` package for other optimizers
* Data augmentation: You should use the `torchvision.transforms` module to try adding random resized crops and horizontal flips of the input data. Check `transforms.RandomResizedCrop` and `transforms.RandomHorizontalFlip` for this. Feel free to apply more [transforms](https://pytorch.org/docs/stable/torchvision/transforms.html) for data augmentation which can lead to better performance. 
* Epochs: Once you have found a generally good hyperparameter setting try training for more epochs
* Loss function: You might want to add weighting to the `MultiLabelSoftMarginLoss` for classes that are less well represented or experiment with a different loss function



#### Note
We will soon be providing some initial expectations of mAP values as a function of epoch so you can get an early idea whether your implementation works without waiting a long time for training to converge.

### What to submit 
Submit your best model to Kaggle and save all plots for the writeup.


In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:


normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std= [0.229, 0.224, 0.225])

train_transform = transforms.Compose([
            #torchvision.transforms.ColorJitter(hue=.1, saturation=.05),
            torchvision.transforms.RandomHorizontalFlip(),
            torchvision.transforms.RandomRotation(10, resample=PIL.Image.BILINEAR),
            transforms.Resize(227),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            normalize
        ])

test_transform = transforms.Compose([
            transforms.Resize(227),
            transforms.CenterCrop(227),
            transforms.ToTensor(),
            normalize,
        ])



In [9]:
#Randomize the order of the input images
#s=np.arange(ds_train.names.shape[0])
s=np.arange(39209)
#np.random.seed(43)
#np.random.shuffle(s)

s_train, s_val = train_test_split(s, test_size=0.25, random_state=1)
#s_train, s_test = train_test_split(s_train, test_size=0.25, random_state=1)


ds_train = MyDataset(path,'Train',train_transform, s_train)
ds_val = MyDataset(path,'Train',train_transform, s_val)




s=np.arange(12630)
np.random.seed(43)
np.random.shuffle(s)
ds_test = MyDataset(path,'Test',train_transform, s)
print(ds_train.names.shape)
print(ds_val.names.shape)
print(ds_test.names.shape)




(29406,)
(9803,)
(12630,)


In [10]:
num_epochs = 20
test_frequency = 5
batch_size = 64

train_loader = torch.utils.data.DataLoader(dataset=ds_train,
                                               batch_size=batch_size, 
                                               shuffle=True,
                                               num_workers=1)

val_loader = torch.utils.data.DataLoader(dataset=ds_val,
                                               batch_size=batch_size, 
                                               shuffle=True,
                                               num_workers=1)

test_loader = torch.utils.data.DataLoader(dataset=ds_test,
                                               batch_size=batch_size, 
                                               shuffle=False,
                                               num_workers=1)

In [11]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

In [14]:
criterion = nn.MultiLabelSoftMarginLoss()

In [15]:
# Load Pretrained AlexNet
classifier = torchvision.models.alexnet(pretrained=True)
classifier.classifier._modules['6'] = nn.Linear(4096, num_classes)   
classifier = classifier.to(device)
optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9)
criterion = nn.MultiLabelSoftMarginLoss()

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth


In [16]:
classifier.load_state_dict(torch.load('./classifier.pth'))

<All keys matched successfully>

In [ ]:
# TODO: Run your own classifier here
#classifier = SimpleClassifier().to(device)





#optimizer = torch.optim.SGD(classifier.parameters(), lr=0.01, momentum=0.9, weight_decay = 0.9)
#optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)


#decayRate = 0.97
#lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)
# optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-4)

classifier, train_losses, val_losses, train_mAPs, val_mAPs = train(classifier, num_epochs, train_loader, val_loader, criterion, optimizer, test_frequency)

torch.save(classifier.state_dict(), './classifier.pth')

In [ ]:
plot_losses(train_losses, val_losses, test_frequency, num_epochs)
plot_mAP(train_mAPs, val_mAPs, test_frequency, num_epochs)

In [21]:
import pickle
with open('./results.pkl', 'wb') as f:
    pickle.dump([mAP_test, test_loss, test_aps, classifier], f)


In [17]:
filehandler = open("./results2.pkl","wb")
pickle.dump([mAP_test, test_loss, test_aps, classifier],filehandler)
filehandler.close()

In [12]:
import pickle
#with open(r"./results.pkl", "rb") as input_file:
file = open("./results.pkl",'rb')
mAP_test, test_loss, test_aps, classifier = pickle.load(file)
file.close()

In [15]:
mAP_test, test_loss, test_aps = test_classifier(test_loader, classifier, criterion)
print(mAP_test)

-------  Class: 1                AP:   0.9981  -------
-------  Class: 2                AP:   0.9948  -------
-------  Class: 3                AP:   0.9912  -------
-------  Class: 4                AP:   0.9899  -------
-------  Class: 5                AP:   0.9878  -------
-------  Class: 6                AP:   0.9983  -------
-------  Class: 7                AP:   0.9983  -------
-------  Class: 8                AP:   0.9923  -------
-------  Class: 9                AP:   1.0000  -------
-------  Class: 10               AP:   1.0000  -------
-------  Class: 11               AP:   0.9914  -------
-------  Class: 12               AP:   0.9997  -------
-------  Class: 13               AP:   0.9998  -------
-------  Class: 14               AP:   1.0000  -------
-------  Class: 15               AP:   0.9986  -------
-------  Class: 16               AP:   1.0000  -------
-------  Class: 17               AP:   0.9943  -------
-------  Class: 18               AP:   0.9924  -------
-------  C

In [ ]:
torch.save(classifier.state_dict(), './voc_my_best_classifier.pth')
output_submission_csv('my_solution.csv', test_aps)